In [ ]:
!pip3 install scipy

In [1]:
#讀取TRT模型

import os
import time

import pycuda.autoinit
from utils.yolo_classes import get_cls_dict
from utils.display import open_window, set_display, show_fps
from utils.visualization import BBoxVisualization
from utils.yolo import TRT_YOLO
trt_yolo = TRT_YOLO("yolov4-tiny-224", (224, 224), 4)

In [8]:
import cv2
import numpy as np
from jetbot import Camera
import ipywidgets as widgets
from IPython.display import display
from jetbot import bgr8_to_jpeg
from scipy.stats import linregress
# Function to calculate the distance between two points
SIGN_INDEX_REVERSED = {
    0: 'Maximum Speed Limit',
    1: 'Minimum Speed Limit',
    2: 'Pedestrian Crossing',
    3: 'Railroad Crossing Warning',
    4: 'Right Turn',
    5: 'Stop',
    6: 'Stop and Proceed'
}

left_speed,right_speed =0,0
def getNearest(signs):
    # Find the index of the sign with the largest width
    largest_sign = max(signs, key=lambda x: x)
    # Assuming that the signs array is paired with (width, class_id), return the sign data (e.g., class id and width)
    return largest_sign
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
# 使用線性回歸計算左右車道線的參數
def calculate_line_params(lines):
    if len(lines) == 0:
        return None, None

    # 收集所有点的坐标
    x_coords = []
    y_coords = []
    for x1, y1, x2, y2 in lines:
        x_coords.extend([x1, x2])
        y_coords.extend([y1, y2])

    # 转换为 NumPy 数组
    x_coords = np.array(x_coords)
    y_coords = np.array(y_coords)

    # 构建设计矩阵 A
    A = np.vstack([x_coords, np.ones(len(x_coords))]).T

    # 使用最小二乘法计算斜率和截距
    # 等价于求解 (A.T @ A) @ [slope, intercept] = A.T @ y_coords
    slope, intercept = np.linalg.lstsq(A, y_coords, rcond=None)[0]

    return slope, intercept

# 繪製擬合出的車道線
def draw_lane(image, slope, intercept, color, thickness=5):
    if slope is not None and intercept is not None:
        height, width = image.shape[:2]
        y1 = int(height * 0.6)  # 車道線起點高度（比例）
        y2 = height  # 車道線終點（底部）

        # 防止除以零
        if abs(slope) < 1e-6:
            return
        
        # 計算初始 x1 和 x2
        x1 = int((y1 - intercept) / slope)
        x2 = int((y2 - intercept) / slope)

        # 確保座標在影像範圍內
        def clip_line(x, y):
            """裁剪線段的 x, y 值使其在影像邊界內"""
            if x < 0:  # 超出左邊界
                x = 0
                y = int(slope * x + intercept)
            elif x >= width:  # 超出右邊界
                x = width - 1
                y = int(slope * x + intercept)
            if y < 0:  # 超出上邊界
                y = 0
                x = int((y - intercept) / slope)
            elif y >= height:  # 超出下邊界
                y = height - 1
                x = int((y - intercept) / slope)
            return x, y

        # 裁剪起點和終點
        x1, y1 = clip_line(x1, y1)
        x2, y2 = clip_line(x2, y2)

        # 繪製有效線段
        if 0 <= x1 < width and 0 <= x2 < width and 0 <= y1 < height and 0 <= y2 < height:
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)
        else:
            print(f"Skipping invalid line with coordinates after clipping: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

def process_and_update_image(change):
    global left_speed, right_speed, robot, count, speed_left, speed_right, stop_ignore, railway_ignore
    ALERT_WIDTH = 50

    # 取得新的影像
    img = change['new']
    original_image = img.copy()
    
     # ======= YOLO 物件檢測部分 =======
    boxes, confs, clss = trt_yolo.detect(img)

    # Process detected signs
    signs = []
    for box, cls in zip(boxes, clss):
        width = box[2] - box[0]
        signs.append([width, cls])

        # Draw bounding box and label
        x1, y1, x2, y2 = map(int, box)
        label = SIGN_INDEX_REVERSED[int(cls)]
        cv2.rectangle(original_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(original_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Find and process the nearest sign
    if signs:
        sign = getNearest(signs)
        if sign[1] == 2 and sign[0] > ALERT_WIDTH:
            print('Action: Slow down')
            # robot.set_motors(speed_left * 0.7, speed_right * 0.7)
        elif sign[1] == 3 and sign[0] > ALERT_WIDTH:
            print('Action: Road close')
            # robot.stop()
        elif sign[1] == 2 and sign[0] < ALERT_WIDTH:
            print('Action: Stop')
            # robot.stop()
        elif sign[1] == 1 and sign[0] > (ALERT_WIDTH - 20):
            print('Action: Railway crossing')
            # railway_ignore += 10
            # robot.stop()
            # time.sleep(3)
        prediction_label.value = (
            f"SIGN: {SIGN_INDEX_REVERSED[int(sign[1])]}"
        )

    # ======= 車道線檢測部分 =======
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    edges = cv2.Canny(blurred, 100, 140)
    kernel = np.ones((3, 3), np.uint8)
    morphed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    height, width = morphed.shape
    roi_mask = np.zeros_like(morphed)
    roi_mask[int(height * 0.6):] = 255
    morphed_roi = cv2.bitwise_and(morphed, roi_mask)

    lines = cv2.HoughLinesP(morphed_roi, 1, np.pi / 180, 10, minLineLength=5, maxLineGap=20)
    left_lines, right_lines = [], []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1 + 1e-6)
            if slope < -0.5:  # 左車道線
                left_lines.append((x1, y1, x2, y2))
            elif slope > 0.5:  # 右車道線
                right_lines.append((x1, y1, x2, y2))

    left_slope, left_intercept = calculate_line_params(left_lines)
    right_slope, right_intercept = calculate_line_params(right_lines)

    # 繪製車道線
    draw_lane(original_image, left_slope, left_intercept, (255, 0, 0))  # 藍色
    draw_lane(original_image, right_slope, right_intercept, (0, 255, 0))  # 綠色
    
   

    # ======= 更新影像顯示 =======
    _, jpeg_image = cv2.imencode('.jpg', original_image)
    image_widget.value = jpeg_image.tobytes()
image_widget = widgets.Image(format='jpeg', width=300, height=300)
prediction_label = widgets.Label(value="Prediction: Waiting for data...")
display(image_widget, prediction_label)

Image(value=b'', format='jpeg', height='300', width='300')

Label(value='Prediction: Waiting for data...')

# 直線演算法

In [9]:
import cv2
import numpy as np
from jetbot import Camera
import ipywidgets as widgets
from jetbot import bgr8_to_jpeg
import time
from jetbot import Robot
# Start the camera
from jetbot import Camera
camera = Camera.instance(width=224, height=224,fps=10)
while(1):
    #process_image({'new':camera.value})
    process_and_update_image({'new':camera.value})

Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Road close
Action: Road close
Action: Road close
Action: Road close
Action: Road close
Action: Road close
Action: Road close
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Stop
Action: Slow down
Action: Slow down
Action: Slow down
Action: Slow down
Action: Slow down
Action: Sl

KeyboardInterrupt: 

In [10]:
from jetbot import Camera
camera.stop()

In [ ]:
camera.observe(process_and_update_image, names='value')